In [1]:
!gcloud config set project  bwh-midrc-rapid-res-1655321320

Updated property [core/project].


In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
  project_name = "bwh-midrc-rapid-res-1655321320"
  bucket_name = "midrc-analysis-bwh"
  bucket_path = "bpr-results/"

In [4]:
#install nrrd
!pip install pynrrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 9.5 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


In [5]:
!pip install SimpleITK

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52.8 MB 91 kB/s 


In [6]:
!pip install pyradiomics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 4.9 MB/s 
     |████████████████████████████████| 109 kB 40.4 MB/s 
     |████████████████████████████████| 500 kB 43.2 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13724 sha256=b7439f0cc06bcd80f3b926a175e93889d43bb5cc7e42b8d27635be010ecf29a6
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [8]:
%%capture
!pip install dicomweb-client

In [9]:
from __future__ import print_function
import six

import os
import glob
import numpy as np
import shutil
from google.cloud import storage
import nrrd
import SimpleITK as sitk
import json
import matplotlib.pyplot as plt
from matplotlib import transforms
from matplotlib.colors import ListedColormap

import radiomics
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

import csv
import pandas as pd

In [10]:
%%bigquery --project=bwh-midrc-rapid-res-1655321320 ct_limited_open_a1_r1

WITH
  nlst_instances_per_series AS (
    SELECT
      DISTINCT(StudyInstanceUID),
      SeriesInstanceUID,
      COUNT(DISTINCT(SOPInstanceUID)) AS num_instances,
      COUNT(DISTINCT(ARRAY_TO_STRING(ImagePositionPatient,"/"))) AS position_count,
      COUNT(DISTINCT(ARRAY_TO_STRING(ImageOrientationPatient,"/"))) AS orientation_count,
      MIN(SAFE_CAST(SliceThickness AS float64)) AS min_SliceThickness,
      MAX(SAFE_CAST(SliceThickness AS float64)) AS max_SliceThickness,
      MIN(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as min_SliceLocation, 
      MAX(SAFE_CAST(ImagePositionPatient[SAFE_OFFSET(2)] AS float64)) as max_SliceLocation,
      STRING_AGG(DISTINCT(SAFE_CAST("LOCALIZER" IN UNNEST(ImageType) AS string)),"") AS has_localizer
    FROM
      bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all
    WHERE
      (collection_id = "Open-R1" or collection_id = "Open-A1") and Modality = "CT"
    GROUP BY
      StudyInstanceUID,
      SeriesInstanceUID
      ), 
  nlst_values_per_series AS (
    SELECT 
    ANY_VALUE(dicom_all.PatientID) AS PatientID,
    dicom_all.SeriesInstanceUID,
    ANY_VALUE(nlst_instances_per_series.num_instances) AS num_instances,
    ANY_VALUE(nlst_instances_per_series.max_SliceThickness) AS SliceThickness,
    ANY_VALUE((nlst_instances_per_series.max_SliceLocation - nlst_instances_per_series.min_SliceLocation)) AS PatientHeightScanned
  FROM
    bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
  JOIN
    nlst_instances_per_series
  ON
    dicom_all.SeriesInstanceUID = nlst_instances_per_series.SeriesInstanceUID
  WHERE
    min_SliceThickness >= 1.5 
    AND max_SliceThickness <= 3.5 
    AND nlst_instances_per_series.num_instances > 100
    AND nlst_instances_per_series.num_instances/nlst_instances_per_series.position_count = 1
    AND nlst_instances_per_series.orientation_count = 1
    AND has_localizer = "false"
  GROUP BY
    SeriesInstanceUID
  )
  SELECT 
    dicom_all.PatientID,
    dicom_all.StudyInstanceUID,
    dicom_all.SeriesInstanceUID,
    dicom_all.SOPInstanceUID,
    dicom_all.collection_id,
    dicom_all.PatientAge,
    dicom_all.PatientWeight,
    nlst_values_per_series.num_instances,
    nlst_values_per_series.SliceThickness,
    nlst_values_per_series.PatientHeightScanned
  FROM
    bwh-midrc-rapid-res-1655321320.midrc_dicom_us.dicom_all AS dicom_all
  JOIN
    nlst_values_per_series 
  ON
    dicom_all.SeriesInstanceUID = nlst_values_per_series.SeriesInstanceUID

In [11]:
def download_series(study_instance_uid, series_instance_uid, sop_instance_uids, dest_dir):
  import pydicom
  token = !gcloud auth print-access-token
  token = token[0]

  PROJECT_ID="bwh-midrc-rapid-res-1655321320"
  REGION="us-central1"

  DATASET_ID="midrc"
  DICOM_STORE_ID="midrc-dicom"
  
  my_project = "bwh-midrc-rapid-res-1655321320"
  location = "us-central1"
  dataset_id = "midrc"
  dicom_store_id = "midrc-dicom"

  url = f"https://healthcare.googleapis.com/v1/projects/{my_project}/locations/{location}/datasets/{dataset_id}/dicomStores/{dicom_store_id}/dicomWeb"
  headers = {
      "Authorization" : "Bearer %s" % token
  }

  import dicomweb_client

  client = dicomweb_client.api.DICOMwebClient(url, headers=headers)

  idx=0
  for sop_instance_uid in sop_instance_uids:
    retrievedInstance = client.retrieve_instance(
                study_instance_uid=study_instance_uid,
                series_instance_uid=series_instance_uid,
                sop_instance_uid=sop_instance_uid)
    pydicom.filewriter.write_file(f"{dest_dir}/file{idx}.dcm", retrievedInstance)
    idx+=1

In [12]:
def file_exists_in_bucket(project_name, bucket_name, file_gs_uri):
  
  """
  Check whether a file exists in the specified Google Cloud Storage Bucket.

  Arguments:
    project_name : required - name of the GCP project.
    bucket_name  : required - name of the bucket (without gs://)
    file_gs_uri  : required - file GS URI
  
  Returns:
    file_exists : boolean variable, True if the file exists in the specified,
                  bucket, at the specified location; False if it doesn't.

  Outputs:
    This function [...]
  """

  storage_client = storage.Client(project = project_name)
  bucket = storage_client.get_bucket(bucket_name)
  
  bucket_gs_url = "gs://%s/"%(bucket_name)
  path_to_file_relative = file_gs_uri.split(bucket_gs_url)[-1]

  print("Searching %s for: \n%s\n"%(bucket_gs_url, path_to_file_relative))

  file_exists = bucket.blob(path_to_file_relative).exists(storage_client)
  
  return file_exists

In [13]:
import logging
urllib3_logger = logging.getLogger('urllib3')
urllib3_logger.setLevel(logging.CRITICAL) # suppress messages upon download

In [ ]:
from google.cloud import storage

storage_client = storage.Client(project = project_name)
#bucket = storage_client.get_bucket(bucket_name)

series_instance_uids = []

#blobs = storage_client.list_blobs(bucket)
blobs = storage_client.list_blobs(bucket_name, prefix=bucket_path) #, delimiter='/') #don't search in non-axial sub-folder
for blob in blobs:
    bn = blob.name 
    # ex: bpr-results/1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334.json
    bs = bn.split('.')
    if bs[-1] == 'json':
      #print(bn)
      bss = bn.split('/')[1]
      #print(bss)
      bs3 = bss.split('.')
      bs4 = '.'.join(bs3[:-1])
      series_instance_uids.append(bs4)

print(series_instance_uids[:10])

['1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334', '1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715', '1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952', '1.2.826.0.1.3680043.10.474.419639.189346812051260775638656981947', '1.2.826.0.1.3680043.10.474.419639.192916356998524553834723357563', '1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063', '1.2.826.0.1.3680043.10.474.419639.249044315484665760654506668895', '1.2.826.0.1.3680043.10.474.419639.259584978733948574762940092562', '1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419', '1.2.826.0.1.3680043.10.474.419639.272117657178318732150423166273']


In [ ]:
print(len(series_instance_uids))

1196


In [14]:
from google.cloud import storage

storage_client = storage.Client(project = project_name)
#bucket = storage_client.get_bucket(bucket_name)

series_instance_uids = []

#blobs = storage_client.list_blobs(bucket)
blobs = storage_client.list_blobs(bucket_name, prefix=bucket_path, delimiter='/')
for blob in blobs:
    bn = blob.name 
    # ex: bpr-results/1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334.json
    (r, ext) = os.path.splitext(bn)
    #print(r,ext)
    if ext == '.json':
      u = os.path.split(r)[-1]
      #print(u)
      series_instance_uids.append(u)

print(series_instance_uids[:10])

['1.2.826.0.1.3680043.10.474.419639.105799060738901793068313281334', '1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715', '1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952', '1.2.826.0.1.3680043.10.474.419639.189346812051260775638656981947', '1.2.826.0.1.3680043.10.474.419639.192916356998524553834723357563', '1.2.826.0.1.3680043.10.474.419639.198735019931123383691750806063', '1.2.826.0.1.3680043.10.474.419639.249044315484665760654506668895', '1.2.826.0.1.3680043.10.474.419639.259584978733948574762940092562', '1.2.826.0.1.3680043.10.474.419639.269534881585852761235289087419', '1.2.826.0.1.3680043.10.474.419639.272117657178318732150423166273']


In [ ]:
print(len(series_instance_uids))

495


In [ ]:
df = ct_limited_open_a1_r1
k=0

good_seriesuids = []

#series_instance_uids = ['1.2.826.0.1.3680043.10.474.419639.106364025147079899289440200715']

for seriesuid in series_instance_uids:
  #print(seriesuid)
  studyuids = list(set(df[df['SeriesInstanceUID']==seriesuid]['StudyInstanceUID'].tolist()))
  if len(studyuids)>0:
    k=k+1
    #print(seriesuid)
    studyuid = studyuids[0]
    sops = df[ (df['StudyInstanceUID']==studyuid) & (df['SeriesInstanceUID']==seriesuid) ]['SOPInstanceUID'].tolist()
    print(len(sops))
    #download_series(studyuid, seriesuid, sops, path_downloaded)
    good_seriesuids.append(seriesuid)
print(k)

In [ ]:
# Make a list of seriesuid's in the bucket
storage_client = storage.Client()
blobs = storage_client.list_blobs(bucket_name)
s = []
k=0
for b in blobs:
  #print('raw', b.name)
  p = os.path.split(b.name)
  #print(p)
  # skip non-axial and folders
  # temporarily autoremove any slashes from bucket_path
  if p[0]==bucket_path.replace('/','') and p[1].split('.')[-1]=='nrrd':
    k +=1
    seriesuid_tmp = '.'.join(p[1].split('.')[:-1])
    s.append(seriesuid_tmp)
    if k%10==0:
      print(k,' ', end='')
    if k%120==0:
        print('')
seriesuid_list = list(set([x.split('_')[1] for x in s ]))
print('\n',len(seriesuid_list))

10  20  30  40  50  60  70  80  90  100  110  120  
130  140  150  160  170  180  190  200  210  220  230  240  
250  260  270  280  290  300  310  320  330  340  350  360  
370  380  390  400  410  420  430  440  450  460  470  480  
490  500  510  520  530  540  550  560  570  580  590  600  
610  620  630  640  650  660  670  680  690  700  710  720  
730  740  750  760  770  780  790  800  810  820  830  840  
850  860  870  880  890  900  910  920  930  940  950  960  
970  980  
 494


In [34]:
# upload files:
# - parameter file for radiomics 
# - meta json for segmentation
# - segment code mapping
# - shape feature code mapping

from google.colab import files
uploaded = files.upload()

Saving shape_features_code_mapping.csv to shape_features_code_mapping.csv


In [33]:
# Processing series

path_downloaded = '/content/downloaded_data'
path_nifti = '/content/nifti_data'
path_json =  '/content/json_data'
path_nrrd = '/content/nrrd_data'
path_dcm = '/content/dcm_data'

good_seriesuids = ['1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952']

#good_seriesuids = [seriesuid_list[0]]

k=0
for seriesuid in good_seriesuids:
  k+=1
  print(k)

  # clean up from previous iterations and recreate temp directories
  if 1:
    for x in [path_downloaded, path_nifti, path_json, path_nrrd, path_dcm]:
      if os.path.isdir(x):
        try:
          shutil.rmtree(x)
        except OSError as err:
          print("Error: %s : %s" % (x, err.strerror))  
      os.mkdir(x)

  file_gs_uri_seg = "gs://%s/%sseg_%s.nrrd" % (bucket_name, bucket_path, seriesuid) # "/"  exists in the path
  file_gs_uri_ct = "gs://%s/%sct_%s.nrrd" % (bucket_name, bucket_path, seriesuid)
  file_gs_uri_dcmseg = "gs://%s/%sdcmseg_%s" % (bucket_name, bucket_path, seriesuid)
  
  cmd1 = "gsutil cp %s %s" % (file_gs_uri_seg, path_nrrd)
  print(cmd1)
  os.system(cmd1)
  
  cmd2 = "gsutil cp %s %s" % (file_gs_uri_ct, path_nrrd)
  print(cmd2)
  os.system(cmd2)
  
  cmd3 = "gsutil cp %s %s" % (file_gs_uri_dcmseg, path_dcm)
  print(cmd3)
  os.system(cmd3)

  fn_seg_nrrd = os.path.join(path_nrrd, 'seg_%s.nrrd' % seriesuid)
  fn_ct_nrrd = os.path.join(path_nrrd, 'ct_%s.nrrd' % seriesuid)
  fn_param = os.path.join('/content','param_ct_shape.yaml')

  # Instantiate the extractor
  extractor = featureextractor.RadiomicsFeatureExtractor(fn_param)

  # lung masks
  lungs = fn_seg_nrrd
  lungs, lungsheader = nrrd.read(lungs)

  right_lung = np.zeros(lungs.shape)
  temp =np.where(lungs==1) 
  right_lung[temp]=1

  left_lung = np.zeros(lungs.shape)
  temp =np.where(lungs==2) 
  left_lung[temp]=1

  fn_right_lung = os.path.join(path_nrrd, 'rlung_%s.nrrd' % seriesuid)
  fn_left_lung =  os.path.join(path_nrrd, 'llung_%s.nrrd' % seriesuid)

  nrrd.write(fn_right_lung, right_lung, header = lungsheader)
  nrrd.write(fn_left_lung, left_lung, header = lungsheader)

  # process the lungs
  result = extractor.execute(fn_ct_nrrd, fn_right_lung)
  for key, val in six.iteritems(result):
    print("\t%s: %s" %(key, val))
  fn_out = '/content/radiomics_features_right_lung_%s.csv' % seriesuid
  df = pd.DataFrame.from_dict(six.iteritems(result))# columns = ['Feature','Value'])
  df.to_csv(fn_out, index=False)

  result = extractor.execute(fn_ct_nrrd, fn_left_lung)
  for key, val in six.iteritems(result):
    print("\t%s: %s" %(key, val))
  fn_out = '/content/radiomics_features_left_lung_%s.csv' % seriesuid
  df = pd.DataFrame.from_dict(six.iteritems(result))# columns = ['Feature','Value'])
  df.to_csv(fn_out, index=False)



1
gsutil cp gs://midrc-analysis-bwh/bpr-results/seg_1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952.nrrd /content/nrrd_data
gsutil cp gs://midrc-analysis-bwh/bpr-results/ct_1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952.nrrd /content/nrrd_data
gsutil cp gs://midrc-analysis-bwh/bpr-results/dcmseg_1.2.826.0.1.3680043.10.474.419639.149051607502633615235577977952 /content/dcm_data


INFO:radiomics.featureextractor:Loading parameter file /content/param_ct_shape.yaml
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask
INFO:radiomics.imageoperations:Applying resampling from spacing [0.703125 0.703125 2.5     ] and size [512 512 145] to spacing [1. 1. 1.] and size [211, 232, 309]
INFO:radiomics.featureextractor:Computing shape
Feature Compactness1 is deprecated, use with caution!
Feature Compactness2 is deprecated, use with caution!
Feature SphericalDisproportion is deprecated, use with caution!
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Adding image type "LoG" with custom settings: {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}
INFO:radiomics.featureextractor:Adding image type "Wavelet" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.imageoperations:Computing LoG with sig

	diagnostics_Versions_PyRadiomics: v3.0.1
	diagnostics_Versions_Numpy: 1.21.6
	diagnostics_Versions_SimpleITK: 2.2.0
	diagnostics_Versions_PyWavelet: 1.3.0
	diagnostics_Versions_Python: 3.7.15
	diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [1, 1, 1], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 10, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'voxelArrayShift': 1000}
	diagnostics_Configuration_EnabledImageTypes: {'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}, 'Wavelet': {}}
	diagnostics_Image-original_Hash: 0cc689e5cf485b8d72435a310b68b06a61674721
	diagnostics_Image-original_Dimensionality: 3D
	diagnostics_Image-original_Spacing: (0.703125, 0.703125, 2.5)
	diagnostics_Image-original_Size: (512, 512, 145)
	diagnostics_Image-original_Mea

INFO:radiomics.imageoperations:Applying resampling from spacing [0.703125 0.703125 2.5     ] and size [512 512 145] to spacing [1. 1. 1.] and size [156, 215, 232]
INFO:radiomics.featureextractor:Computing shape
Feature Compactness1 is deprecated, use with caution!
Feature Compactness2 is deprecated, use with caution!
Feature SphericalDisproportion is deprecated, use with caution!
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Adding image type "LoG" with custom settings: {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}
INFO:radiomics.featureextractor:Adding image type "Wavelet" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.imageoperations:Computing LoG with sigma 1
INFO:radiomics.featureextractor:Calculating features for log-sigma-1-0-mm-3D image
INFO:radiomics.imageoperations:Computing LoG with sigma 2
INFO:radiomics.featureextractor:Calculating features for log

	diagnostics_Versions_PyRadiomics: v3.0.1
	diagnostics_Versions_Numpy: 1.21.6
	diagnostics_Versions_SimpleITK: 2.2.0
	diagnostics_Versions_PyWavelet: 1.3.0
	diagnostics_Versions_Python: 3.7.15
	diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [1, 1, 1], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 10, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'voxelArrayShift': 1000}
	diagnostics_Configuration_EnabledImageTypes: {'Original': {}, 'LoG': {'sigma': [1.0, 2.0, 3.0, 4.0, 5.0]}, 'Wavelet': {}}
	diagnostics_Image-original_Hash: 0cc689e5cf485b8d72435a310b68b06a61674721
	diagnostics_Image-original_Dimensionality: 3D
	diagnostics_Image-original_Spacing: (0.703125, 0.703125, 2.5)
	diagnostics_Image-original_Size: (512, 512, 145)
	diagnostics_Image-original_Mea

In [25]:
!pip uninstall highdicom
!git clone https://github.com/herrmannlab/highdicom.git
#!cd highdicom && python setup.py install
!cd highdicom && pip install .

!pip install pydicom


Cloning into 'highdicom'...
remote: Enumerating objects: 5600, done.
remote: Counting objects: 100% (2150/2150), done.
remote: Compressing objects: 100% (543/543), done.
remote: Total 5600 (delta 1838), reused 1710 (delta 1536), pack-reused 3450
Receiving objects: 100% (5600/5600), 3.07 MiB | 7.73 MiB/s, done.
Resolving deltas: 100% (3641/3641), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/highdicom
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 339 kB 5.3 MB/s 
     |████████████████████████████████| 4.3 MB 35.7 MB/s 
     |

In [26]:
# Packages for the structured report 

import highdicom

from pathlib import Path

import highdicom as hd

from pydicom.uid import generate_uid
from pydicom.filereader import dcmread
from pydicom.sr.codedict import codes

from highdicom.sr.content import (
    FindingSite,
    ImageRegion,
    ImageRegion3D,
    SourceImageForRegion,
    SourceImageForMeasurement,
    SourceImageForMeasurementGroup
)
from highdicom.sr.enum import GraphicTypeValues3D
from highdicom.sr.enum import GraphicTypeValues
from highdicom.sr.sop import Comprehensive3DSR, ComprehensiveSR
from highdicom.sr.templates import (
    DeviceObserverIdentifyingAttributes,
    Measurement,
    MeasurementProperties,
    MeasurementReport,
    MeasurementsAndQualitativeEvaluations,
    ObservationContext,
    ObserverContext,
    PersonObserverIdentifyingAttributes,
    PlanarROIMeasurementsAndQualitativeEvaluations,
    RelationshipTypeValues,
    TrackingIdentifier,
    QualitativeEvaluation,
    ImageLibrary,
    ImageLibraryEntryDescriptors
)
from highdicom.sr.value_types import (
    CodedConcept,
    CodeContentItem,
)

import logging
logger = logging.getLogger("highdicom.sr.sop")
logger.setLevel(logging.INFO)

In [ ]:
def get_label_and_names_from_metadata_json(dicomseg_json):

  """Returns two lists containing the label values and the corresponding
     CodeMeaning values

  Inputs: 
    dicomseg_json : metajson file

  Outputs:
    label_values  : label values from the metajson file 
    label_names   : the corresponding CodeMeaning values 

  """

  f = open(dicomseg_json)
  meta_json = json.load(f)

  print(meta_json)

  num_regions = len(meta_json['segmentAttributes'][0])
  print ('num_regions: ' + str(num_regions))

  label_values = []
  label_names = [] 
  for n in range(0,num_regions):
    # label_values.append(n)
    label_value = meta_json['segmentAttributes'][0][n]['labelID']
    label_name = meta_json['segmentAttributes'][0][n]['SegmentedPropertyTypeCodeSequence']['CodeMeaning']
    label_values.append(label_value)
    label_names.append(label_name)

  return label_values, label_names

In [28]:
def create_structured_report_metajson_for_shape_features(SeriesInstanceUID, 
                                                         SOPInstanceUID_seg,
                                                         seg_file, 
                                                         dcm_directory, 
                                                         segments_code_mapping_df,
                                                         shape_features_code_mapping_df,
                                                         df_features, 
                                                         SegmentAlgorithmName
                                                         ):
  
  """Function that creates the metajson necessary for the creation of a
  structured report from a pandas dataframe of label names and features for 
  each. 

  Inputs: 
    SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT 
                                      file 
    SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file 
    seg_file                        : filename of SEG DCM file 
    dcm_directory                   : ct directory that will be sorted in 
                                      terms of axial ordering according to the 
                                      ImagePositionPatient and ImageOrientation 
                                      fields
    segments_code_mapping_df        : dataframe that holds the names of the 
                                      segments and the associated code values etc.
    shape_features_code_mapping_df  : dataframe that holds the names of the 
                                      features and the associated code values etc. 
    df_features                     : a pandas dataframe holding the segments and a 
                                      set of 3D shape features for each 
    SegmentAlgorithmName            : the name of the algorithm used to create the 
                                      segmentations - e.g. '3d_fullres_tta_nnUnet'

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report 
  """ 

  # --- Get the version number for the pyradiomics package --- #

  pyradiomics_version_number = str(radiomics.__version__)
  
  # --- Sort the dcm files first according to --- # 
  # --- ImagePositionPatient and ImageOrientation --- #

  files_sorted, sop_all_sorted, pos_all_sorted = order_dicom_files_image_position(dcm_directory)
  files_sorted = [os.path.basename(f) for f in files_sorted]

  # --- Create the header for the json --- # 
  
  inputMetadata = {}
  inputMetadata["@schema"]= "https://raw.githubusercontent.com/qiicr/dcmqi/master/doc/schemas/sr-tid1500-schema.json#"
  # inputMetadata["SeriesDescription"] = "Measurements"
  inputMetadata["SeriesDescription"] = SegmentAlgorithmName + '_' + "Measurements"
  inputMetadata["SeriesNumber"] = "1001"
  inputMetadata["InstanceNumber"] = "1"

  inputMetadata["compositeContext"] = [seg_file] # not full path

  inputMetadata["imageLibrary"] = files_sorted # not full path 

   # inputMetadata["observerContext"] = {
  #                                     "ObserverType": "PERSON",
  #                                     "PersonObserverName": "Reader1"
  #                                   }
  # inputMetadata["observerContext"] = {
  #                     "ObserverType": "DEVICE",
  #                     "DeviceObserverName": "pyradiomics",
  #                     "DeviceObserverModelName": "v3.0.1"
  #                   }
  inputMetadata["observerContext"] = {
                      "ObserverType": "DEVICE",
                      "DeviceObserverName": "pyradiomics",
                      "DeviceObserverModelName": pyradiomics_version_number
                    }

  inputMetadata["VerificationFlag"]  = "UNVERIFIED"
  inputMetadata["CompletionFlag"] =  "COMPLETE"
  inputMetadata["activitySession"] = "1"
  inputMetadata["timePoint"] = "1"

  # ------------------------------------------------------------------------- # 
  # --- Create the measurement_dict for each segment - holds all features --- # 

  measurement = [] 

  # --- Now create the dict for all features and all segments --- #

  # --- Loop over the number of segments --- #

  # number of rows in the df_features 
  num_segments = df_features.shape[0]

  # Array of dictionaries - one dictionary for each segment 
  measurement_across_segments_combined = [] 

  for segment_id in range(0,num_segments):

    ReferencedSegment = df_features['ReferencedSegment'].values[segment_id]
    FindingSite = df_features['label_name'].values[segment_id]

    print('segment_id: ' + str(segment_id))
    print('ReferencedSegment: ' + str(ReferencedSegment))
    print('FindingSite: ' + str(FindingSite))

    # --- Create the dict for the Measurements group --- # 
    TrackingIdentifier = "Measurements group " + str(ReferencedSegment)

    segment_row = segments_code_mapping_df[segments_code_mapping_df["segment"] == FindingSite]
    # print(segment_row)
        
    my_dict = {
      "TrackingIdentifier": str(TrackingIdentifier),
      "ReferencedSegment": int(ReferencedSegment),
      "SourceSeriesForImageSegmentation": str(SeriesInstanceUID),
      "segmentationSOPInstanceUID": str(SOPInstanceUID_seg),
      "Finding": {
        "CodeValue": "113343008",
        "CodingSchemeDesignator": "SCT",
        "CodeMeaning": "Organ"
      }, 
      "FindingSite": {
        "CodeValue": str(segment_row["FindingSite_CodeValue"].values[0]),
        "CodingSchemeDesignator": str(segment_row["FindingSite_CodingSchemeDesignator"].values[0]),
        "CodeMeaning": str(segment_row["FindingSite_CodeMeaning"].values[0])
      }
    }

    measurement = []  
    # number of features - number of columns in df_features - 2 (label_name and ReferencedSegment)
    num_values = len(df_features.columns)-2 

    feature_list = df_features.columns[2:] # remove first two 


    # For each measurement per region segment
    for n in range(0,num_values): 
      measurement_dict = {}
      row = df_features.loc[df_features['label_name'] == FindingSite]
      feature_row = shape_features_code_mapping_df.loc[shape_features_code_mapping_df["shape_feature"] == feature_list[n]]
      value = str(np.round(row[feature_list[n]].values[0],3))
      measurement_dict["value"] = value
      measurement_dict["quantity"] = {}
      measurement_dict["quantity"]["CodeValue"] = str(feature_row["quantity_CodeValue"].values[0])
      measurement_dict["quantity"]["CodingSchemeDesignator"] = str(feature_row["quantity_CodingSchemeDesignator"].values[0])
      measurement_dict["quantity"]["CodeMeaning"] = str(feature_row["quantity_CodeMeaning"].values[0])
      measurement_dict["units"] = {}
      measurement_dict["units"]["CodeValue"] = str(feature_row["units_CodeValue"].values[0])
      measurement_dict["units"]["CodingSchemeDesignator"] = str(feature_row["units_CodingSchemeDesignator"].values[0])
      measurement_dict["units"]["CodeMeaning"] = str(feature_row["units_CodeMeaning"].values[0])
      measurement_dict["measurementAlgorithmIdentification"] = {}
      measurement_dict["measurementAlgorithmIdentification"]["AlgorithmName"] = "pyradiomics"
      measurement_dict["measurementAlgorithmIdentification"]["AlgorithmVersion"] = str(pyradiomics_version_number)
      measurement.append(measurement_dict) 

    measurement_combined_dict = {}
    measurement_combined_dict['measurementItems'] = measurement # measurement is an array of dictionaries 

    output_dict_one_segment = {**my_dict, **measurement_combined_dict}

    # append to array for all segments 

    measurement_across_segments_combined.append(output_dict_one_segment)

  # --- Add the measurement data --- # 

  inputMetadata["Measurements"] = {}
  inputMetadata["Measurements"] = measurement_across_segments_combined

  return inputMetadata


In [37]:
fn_seg_dcm = os.path.join(path_dcm, 'dcmseg_%s' % seriesuid)
fn_seg_code_mapping = os.path.join('/content','segments_code_mapping.csv')
fn_feature_code_mapping = os.path.join('/content','shape_features_code_mapping.csv')
fn_seg_meta = os.path.join('/content','lung_seg_meta.json')

seg_code_mapping_df = pd.read_csv(fn_seg_code_mapping)
feature_code_mapping_df = pd.read_csv(fn_feature_code_mapping)


In [40]:
ds = dcmread(fn_seg_dcm)
sop = ds.SOPInstanceUID
print(sop)

1.2.276.0.7230010.3.1.4.481034752.4023.1667497562.213485


In [ ]:
create_structured_report_metajson_for_shape_features(seriesuid, 
                                                         sop,
                                                         fn_seg_dcm, 
                                                         path_downloaded, 
                                                         seg_code_mapping_df,
                                                         feature_code_mapping_df,
                                                         df_features, 
                                                         'lungmask'
                                                         )

In [41]:
def save_structured_report_for_shape_features(SeriesInstanceUID, 
                                              SOPInstanceUID_seg, 
                                              pred_dicomseg_path, 
                                              dicomseg_json_path, 
                                              dcm_directory, 
                                              pred_nifti_path, 
                                              nnunet_base_path, 
                                              ct_nifti_path, 
                                              segments_code_mapping_df,
                                              shape_features_code_mapping_df,
                                              sr_json_path,
                                              sr_path, 
                                              SegmentAlgorithmName
                                              ):
  
  """ This function creates the SR necessary for the nnUNet shape features 

  Inputs: 
  SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT 
                                    file 
  SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file 
  pred_dicomseg_path              : filename of DICOM SEG file 
  dicomseg_json_path              : json file for DICOM SEG file 
  dcm_directory                   : list of ct files that will be sorted in 
                                    terms of axial ordering according to the 
                                    ImagePositionPatient and ImageOrientation 
                                    fields
  pred_nifti_path                 : predictions in nifti format 
  nnunet_base_path                : path to hold the split nifti files 
  ct_nifti_path                   : filename for CT nifti file
  segments_code_mapping_df        : dataframe that holds the names of the 
                                    segments and the associated code values etc.
  shape_features_code_mapping_df  : dataframe that holds the names of the 
                                    features and the associated code values etc. 
  sr_json_path                    : the path that the metajson for the SR for 
                                    the 3D shape features will be saved 
  sr_path                         : the path that the SR for the 3D shape 
                                    features will be saved 
  SegmentAlgorithmName            : the name of the algorithm used to create the 
                                    segmentations - e.g. '3d_fullres_tta_nnUnet'

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report 

  """

  # --- get label values and names from metajson file --- #
  label_values, label_names = get_label_and_names_from_metadata_json(dicomseg_json_path)

  # --- split the multilabel nifti into individual files --- #
  split_pred_nii_path = os.path.join(nnunet_base_path, "split_nii")
  if not os.path.isdir(split_pred_nii_path): 
    os.mkdir(split_pred_nii_path)
  split_nii(pred_nifti_path, split_pred_nii_path, label_names)

  # --- compute features and save csv for each region --- #
  if not os.path.isdir(features_csv_path_nnunet):
    os.mkdir(features_csv_path_nnunet) 
  df_features = compute_pyradiomics_3D_features(ct_nifti_path, 
                                                label_values, 
                                                label_names,
                                                split_pred_nii_path, 
                                                nnunet_shape_features_code_mapping_df)
  print ('created df_features')
  
  # --- upload csv file to bucket --- #
  # !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_features_csv_path $gs_uri_features_csv_file

  # remove nii files after saving out pyradiomics results
  !rm $split_pred_nii_path/*
  # remove csv 
  # !rm $pred_features_csv_path

  # --- Create the final metadata for the SR --- #
  inputMetadata = create_structured_report_metajson_for_shape_features(SeriesInstanceUID, 
                                                                       SOPInstanceUID_seg,
                                                                       pred_dicomseg_path, 
                                                                       dcm_directory, 
                                                                       nnunet_segments_code_mapping_df, 
                                                                       nnunet_shape_features_code_mapping_df,
                                                                       df_features, 
                                                                       SegmentAlgorithmName)

  print ('created SR json for shape features')

  # --- Write out json --- #

  with open(sr_json_path, 'w') as f:
    json.dump(inputMetadata, f, indent=2)
  print ('wrote out json for shape features')

  # --- Save the SR for nnUNet shape features --- # 
  # inputImageLibraryDirectory = os.path.join("/content", "raw")
  # outputDICOM = os.path.join("/content","features_sr.dcm")
  # inputCompositeContextDirectory = os.path.join("/content","seg")
  inputImageLibraryDirectory = dcm_directory
  # outputDICOM = sr_json_path
  outputDICOM = sr_path
  # the name of the folder where the seg files are located 
  inputCompositeContextDirectory = os.path.basename(pred_dicomseg_path) # might need to check this
  inputMetadata_json = sr_json_path 

  print ('inputImageLibraryDirectory: ' + str(inputImageLibraryDirectory))
  print ('outputDICOM: ' + str(outputDICOM))
  print ('inputCompositeContextDirectory: ' + str(inputCompositeContextDirectory))
  print ('inputMetadata_json: ' + str(inputMetadata_json)) 
  !tid1500writer --inputImageLibraryDirectory $inputImageLibraryDirectory \
                --outputDICOM $outputDICOM  \
                --inputCompositeContextDirectory $inputCompositeContextDirectory \
                --inputMetadata $inputMetadata_json
  print ('wrote out SR for shape features')

  return
